In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as ConstantKernel
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB

# Data loading and preprocessing

In [ ]:
# load data
data = pd.read_excel('data_scaled.xlsx')
y_index = ['itr']
y = data[y_index]
data.drop('itr', axis=1, inplace=True)
X = data.drop('Interface', axis=1)
# print(X[:10])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# seed everything
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

# XGBoost - all descriptors

In [ ]:
from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBRegressor(learning_rate = 0.2, max_depth = 8, alpha = 50, colsample_bytree = 0.3, objective="reg:linear", random_state=42)

xgb_model.fit(X_train, y_train)

xgb_train_predict_a = xgb_model.predict(X_train)
xgb_valid_predict_a = xgb_model.predict(X_valid)

xgb_train_predict_a.shape = (553,1)
xgb_valid_predict_a.shape = (139,1)

print(xgb_train_predict_a.shape)
print(xgb_valid_predict_a.shape)

R2_train = r2_score(y_train, xgb_train_predict_a)
mse_train = mean_squared_error(xgb_train_predict_a, y_train)
rmse_train = math.sqrt(mse_train)

R2_valid = r2_score(y_valid, xgb_valid_predict_a)
mse_valid = mean_squared_error(xgb_valid_predict_a, y_valid)
rmse_valid = math.sqrt(mse_valid)
    
print('R2_train is %f, R2_valid is %f, mse_train is %f,mse_valid is %f' % (
       R2_train, R2_valid, rmse_train, rmse_valid))

In [ ]:
xgb_model

In [ ]:
from matplotlib import pyplot

importances = xgb_model.feature_importances_
importances = {index: importance for index, importance in zip(X, importances)}
importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)
importances2 = sorted(importances, key=lambda x: x[1], reverse=False)
importances


In [ ]:
import plotly.express as px
rank = pd.DataFrame(importances2, columns=["descriptor", "score"])
print(rank.columns)

fig = px.bar(
    rank, 
    x='score', 
    y="descriptor", 
    orientation='h',  
    width=1200,
    height=1000,
    color='score',
)

fig.update_layout(
    xaxis_tickfont_size=14,
    xaxis=dict(
        title='Score (importance)',
        titlefont_size=22,
        tickfont_size=15,
    ),
    yaxis=dict(
        title='Descriptors',
        titlefont_size=22,
        tickfont_size=15,
    ))

fig.show()

# Select Descriptors

In [ ]:
X_index_selected = [feature for feature, importance in importances[:20]]
X_index_selected.remove('fdensity')
X_index_selected.remove('smass')
X_index_selected.remove('fmass')
print(X_index_selected)
X_train_20 = X_train[X_index_selected]
X_valid_20 = X_valid[X_index_selected]

# XGB with 17 descriptors

In [ ]:
xgb_model = xgb.XGBRegressor(learning_rate = 0.2, max_depth = 9, alpha = 50, colsample_bytree = 0.3, objective="reg:linear", random_state=42)

xgb_model.fit(X_train_20, y_train)

xgb_train_predict = xgb_model.predict(X_train_20)
xgb_valid_predict = xgb_model.predict(X_valid_20)

xgb_train_predict.shape = (553,1)
xgb_valid_predict.shape = (139,1)

print(xgb_train_predict.shape)
print(xgb_valid_predict.shape)

R2_train = r2_score(y_train, xgb_train_predict)
mse_train = mean_squared_error(xgb_train_predict, y_train)
rmse_train = math.sqrt(mse_train)

R2_valid = r2_score(y_valid, xgb_valid_predict)
mse_valid = mean_squared_error(xgb_valid_predict, y_valid)
rmse_valid = math.sqrt(mse_valid)
    
print('R2_train is %f, R2_valid is %f, mse_train is %f,mse_valid is %f' % (
       R2_train, R2_valid, rmse_train, rmse_valid))

In [ ]:
xgb_model

In [ ]:
#load test data
test_data = pd.read_excel('testdata-600K.xlsx')
test_data = test_data[X_index_selected]
print(type(test_data))
print(test_data.shape)


#load constructed systems to append predicted ITR by different models
ms_ITR = pd.read_excel('test data - Copy.xlsx')

In [ ]:
xgb_test_predict = xgb_model.predict(test_data)
print(type(xgb_test_predict))
print(len(xgb_test_predict))
print('finished')
xgb_ITR = pd.DataFrame(xgb_test_predict, columns=['xgb_ITR'])

# Kernel Ridge Regression

## 17 descriptors

In [ ]:
krr = KernelRidge(kernel='rbf', alpha=0.013, gamma=0.1)
krr.fit(X_train_20, y_train)

krr_train_predict = krr.predict(X_train_20)
krr_valid_predict = krr.predict(X_valid_20)
print(krr_train_predict.shape)
print(krr_valid_predict.shape)

R2_train = r2_score(y_train, krr_train_predict)
mse_train = mean_squared_error(krr_train_predict, y_train)
rmse_train = math.sqrt(mse_train)

R2_valid = r2_score(y_valid, krr_valid_predict)
mse_valid = mean_squared_error(krr_valid_predict, y_valid)
rmse_valid = math.sqrt(mse_valid)
    
print('R2_train is %f, R2_valid is %f, mse_train is %f,mse_valid is %f' % (
       R2_train, R2_valid, rmse_train, rmse_valid))

In [ ]:
print(test_data.shape)
krr_test_predict = krr.predict(test_data)
print(type(krr_test_predict))
print(len(krr_test_predict))
print('finished')
krr_ITR = pd.DataFrame(krr_test_predict, columns=['krr_ITR'])

## All descriptors

In [ ]:
krr = KernelRidge(kernel='rbf', alpha=0.025, gamma=0.1)
krr.fit(X_train, y_train)

krr_train_predict_a = krr.predict(X_train)
krr_valid_predict_a = krr.predict(X_valid)
print(krr_train_predict_a.shape)
print(krr_valid_predict_a.shape)

R2_train = r2_score(y_train, krr_train_predict_a)
mse_train = mean_squared_error(krr_train_predict_a, y_train)
rmse_train = math.sqrt(mse_train)

R2_valid = r2_score(y_valid, krr_valid_predict_a)
mse_valid = mean_squared_error(krr_valid_predict_a, y_valid)
rmse_valid = math.sqrt(mse_valid)
    
print('R2_train is %f, R2_valid is %f, mse_train is %f,mse_valid is %f' % (
       R2_train, R2_valid, rmse_train, rmse_valid))

# Deep Neural Network

## 17 descriptors

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
import torch.utils.data as data
import matplotlib.pyplot as plt

seed_everything(42)
train_data = data.TensorDataset(torch.from_numpy(X_train_20.values), torch.from_numpy(y_train.values))
trainloader = data.DataLoader(train_data, batch_size = X_train_20.shape[0], shuffle=True) # just one batch 

train_data2 = torch.from_numpy(X_train_20.values)
valid_data = torch.from_numpy(X_valid_20.values)


#-------------------------------------------------------------------------------
class Model(nn.Module):
    def __init__(self, num_features, dropout):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.linear1 = nn.Linear(num_features, 64)
        
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(dropout)
        self.linear2 = nn.Linear(64, 128)
        
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(dropout)
        self.linear3 = nn.Linear(128, 64)
        
        self.batch_norm4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(dropout)
        self.linear4 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout2(x)
        x = F.relu(self.linear1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.linear2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.linear3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.linear4(x)
        
        return x
        
dropout =  0.25
# hidden_size = 128
learning_rate = 0.0003
Weight_Decay = 5e-5
Epochs = 15000
# Epochs = 8

nn_model = Model(X_train_20.shape[1], dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr= learning_rate, weight_decay=Weight_Decay)

train_loss = []
for epoch in range(Epochs):
   
    for batch in trainloader:
        optimizer.zero_grad()
        
        y_pred = nn_model(batch[0].float())
        loss = criterion(y_pred, batch[1].float())
        if epoch % 100 == 0:
            print('epochs: %d, training loss is : %f' % (epoch, loss))
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
print('training loss:', np.mean(train_loss))
epochs = list(range(Epochs))
plt.plot(epochs, train_loss, 'g', label='Training loss')

plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

nn_model.eval()
nn_y_train_predict = nn_model(train_data2.float())
nn_y_train_predict = nn_y_train_predict.detach().numpy()
nn_y_valid_predict = nn_model(valid_data.float())
nn_y_valid_predict = nn_y_valid_predict.detach().numpy()

#---------------------------------------------------------------------------------
R2_train = r2_score(nn_y_train_predict, y_train)
R2_valid = r2_score(nn_y_valid_predict, y_valid)

mse_train = mean_squared_error(nn_y_train_predict, y_train)   
mse_valid = mean_squared_error(nn_y_valid_predict, y_valid)   
print('R2_train is %f, rmse_train is %f' % (
       R2_train, math.sqrt(mse_train)))
print('R2_valid is %f, rmse_valid is %f' % (
       R2_valid, math.sqrt(mse_valid)))
print(type(nn_y_valid_predict))
print(nn_y_train_predict.shape)
print(nn_y_valid_predict.shape)

In [ ]:
# predict with test data
test_data = test_data.to_numpy(np.float64)
test_data2 = test_data
# test_data = torch.from_numpy(test_data.values)
test_data = torch.from_numpy(test_data).float()


print(type(X_train.values))
print(type(test_data.values))
nn_y_test_predict = nn_model(test_data.float())
nn_y_test_predict = nn_y_test_predict.detach().numpy()
print(type(nn_y_test_predict))
print(nn_y_test_predict)
print(len(nn_y_test_predict))
print('finished')

In [ ]:
dnn_ITR = pd.DataFrame(nn_y_test_predict, columns=['dnn_ITR'])

## All descriptors

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
import torch.utils.data as data
import matplotlib.pyplot as plt

seed_everything(42)
train_data = data.TensorDataset(torch.from_numpy(X_train.values), torch.from_numpy(y_train.values))
trainloader = data.DataLoader(train_data, batch_size = X_train.shape[0], shuffle=True) # just one batch 

train_data2 = torch.from_numpy(X_train.values)
valid_data = torch.from_numpy(X_valid.values)


#-------------------------------------------------------------------------------
class Model(nn.Module):
    def __init__(self, num_features, dropout):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.linear1 = nn.Linear(num_features, 32)
        
        self.batch_norm2 = nn.BatchNorm1d(32)
        self.dropout2 = nn.Dropout(dropout)
        self.linear2 = nn.Linear(32, 64)
        
        self.batch_norm3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(dropout)
        self.linear3 = nn.Linear(64, 32)
        
        self.batch_norm4 = nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(dropout)
        self.linear4 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout2(x)
        x = F.relu(self.linear1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.linear2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.linear3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.linear4(x)
        
        return x
        
dropout =  0.3
# hidden_size = 128
learning_rate = 0.0004
Weight_Decay = 6e-5
Epochs = 15000
# Epochs = 8

nn_model = Model(X_train.shape[1], dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr= learning_rate, weight_decay=Weight_Decay)

train_loss = []
for epoch in range(Epochs):
   
    for batch in trainloader:
        optimizer.zero_grad()
        
        y_pred = nn_model(batch[0].float())
        loss = criterion(y_pred, batch[1].float())
        if epoch % 100 == 0:
            print('epochs: %d, training loss is : %f' % (epoch, loss))
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
print('training loss:', np.mean(train_loss))
epochs = list(range(Epochs))
plt.plot(epochs, train_loss, 'g', label='Training loss')

plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

nn_model.eval()
nn_y_train_predict_a = nn_model(train_data2.float())
nn_y_train_predict_a = nn_y_train_predict_a.detach().numpy()
nn_y_valid_predict_a = nn_model(valid_data.float())
nn_y_valid_predict_a = nn_y_valid_predict_a.detach().numpy()

#---------------------------------------------------------------------------------
R2_train = r2_score(nn_y_train_predict_a, y_train)
R2_valid = r2_score(nn_y_valid_predict_a, y_valid)

mse_train = mean_squared_error(nn_y_train_predict_a, y_train)   
mse_valid = mean_squared_error(nn_y_valid_predict_a, y_valid)   
print('R2_train is %f, rmse_train is %f' % (
       R2_train, math.sqrt(mse_train)))
print('R2_valid is %f, rmse_valid is %f' % (
       R2_valid, math.sqrt(mse_valid)))
print(type(nn_y_valid_predict_a))
print(nn_y_train_predict_a.shape)
print(nn_y_valid_predict_a.shape)

# Ensemble Averaging

## 17 descriptors

In [ ]:
print(krr_train_predict.shape)
print(nn_y_train_predict.shape)
print(xgb_train_predict.shape)
y_pred_train_18 = (krr_train_predict+nn_y_train_predict+xgb_train_predict)/3
y_pred_final_18 = (krr_valid_predict+nn_y_valid_predict+xgb_valid_predict)/3
print(y_pred_train_18.shape)
print(y_pred_final_18.shape)

R2_train = r2_score(y_pred_train_18, y_train)
mse_train = mean_squared_error(y_pred_train_18, y_train) 

R2_valid = r2_score(y_pred_final_18, y_valid)
mse_valid = mean_squared_error(y_pred_final_18, y_valid) 

print('R2_train is %f, rmse_train is %f' % (
       R2_train, math.sqrt(mse_train)))
print('R2_valid is %f, rmse_valid is %f' % (
       R2_valid, math.sqrt(mse_valid)))

## All descriptors

In [ ]:
y_pred_train = (krr_train_predict_a+nn_y_train_predict_a+xgb_train_predict_a)/3
y_pred_final = (krr_valid_predict_a+nn_y_valid_predict_a+xgb_valid_predict_a)/3
print(y_pred_train.shape)
print(y_pred_final.shape)

R2_train = r2_score(y_pred_train, y_train)
mse_train = mean_squared_error(y_pred_train, y_train) 

R2_valid = r2_score(y_pred_final, y_valid)
mse_valid = mean_squared_error(y_pred_final, y_valid) 

print('R2_train is %f, rmse_train is %f' % (
       R2_train, math.sqrt(mse_train)))
print('R2_valid is %f, rmse_valid is %f' % (
       R2_valid, math.sqrt(mse_valid)))

In [ ]:
print('Congratulations! Done!!!')

In [ ]:
print(y_valid)
print(type(y_valid))
print(y_pred_final)
print(type(y_pred_final))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(y_valid, y_pred_final, label='All Descriptors')
ax.scatter(y_valid, y_pred_final_18, label='Selected Descriptors')
x1 = np.linspace(1, 200, 100)
y1 = np.linspace(1, 200, 100)
ax.plot(x1, y1, '--', color='k')
leg = plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Experimental values ($10^{-9} $$m^{2}$K/W)")
plt.ylabel("Experimental values ($10^{-9} $$m^{2}$K/W)")

# plt.scatter(y_valid, y_pred_final)
plt.show()


## Join all ITR predictions

In [ ]:
print(nn_y_test_predict.shape)
xgb_test_predict.shape = (88506,1)
print(xgb_test_predict.shape)
print(krr_test_predict.shape)
average = (nn_y_test_predict+xgb_test_predict+krr_test_predict)/3
en_ITR = pd.DataFrame(average, columns=['en_ITR'])
print(en_ITR.shape)

In [ ]:
ms_ITR = pd.concat([ms_ITR, krr_ITR, xgb_ITR, dnn_ITR, en_ITR], join = 'outer', axis = 1) # combine all ITR with original data
ms_ITR.to_excel("ms_ITR_600K_20210626.xlsx") # write to excel
print('congratulations! All done!')